С сайта https://books.toscrape.com/catalogue/category/books_1/page-1.html собрать название книги, ее стоимость, а также ссылку на книгу

In [1]:
import requests  
url = 'http://books.toscrape.com/catalogue/page-1.html' # site for books parsing, first page
response = requests.get(url)
response

<Response [200]>

In [2]:
response.content[:1000] # get the content of a page

b'\n\n<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:30" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon"

In [3]:
from bs4 import BeautifulSoup

In [4]:
tree = BeautifulSoup(response.content, 'html.parser') # tree of html tags

In [5]:
tree.html.head.title # title tag in head 

<title>
    All products | Books to Scrape - Sandbox
</title>

In [6]:
tree.html.head.title.text # get the text content of tag

'\n    All products | Books to Scrape - Sandbox\n'

In [7]:
tree.find('article', {'class': 'product_pod'}) # find article tag with class=product_pod

<article class="product_pod">
<div class="image_container">
<a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [8]:
tree.find('article', {'class': 'product_pod'}).h3 # find h3 tag in it

<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>

In [9]:
tree.find('article', {'class': 'product_pod'}).find('p', {'class': 'price_color'}) # find p tag with class=price_color in it

<p class="price_color">£51.77</p>

In [10]:
tree.find('article', {'class': 'product_pod'}).find('p', {'class': 'price_color'}).text # get the text content of p tag

'£51.77'

get_page is a function for getting the main info about a book

In [12]:
def get_page(p):
    # p - site link fragment
    # изготовили ссылку
    url = 'http://books.toscrape.com/catalogue/page-{}.html'.format(p)
    
    # сходили по ней
    response = requests.get(url)
    
    # построили дерево 
    tree = BeautifulSoup(response.content, 'html.parser')
    
    # нашли в нём всё самое интересное
    books = tree.find_all('article', {'class' : 'product_pod'})
    
    infa = [ ]
    
    for book in books:
        infa.append({'price': book.find('p', {'class': 'price_color'}).text,
                     'href': book.h3.a.get('href'),
                     'title': book.h3.a.get('title')})
                     
    return infa # list of dictionaries in which we keep book price, title, href

In [13]:
infa = [ ]
# http://books.toscrape.com/catalogue has 50 pages of books, we use range and get_page function to gather info and store it in infa
for p in range(1,51):
    infa.extend(get_page(p))

In [14]:
len(infa)

1000

In [15]:
infa[0] # information about first book on this site

{'price': '£51.77',
 'href': 'a-light-in-the-attic_1000/index.html',
 'title': 'A Light in the Attic'}

In [16]:
import pandas as pd
# put infa into dataframe
df = pd.DataFrame(infa)
df.head()# get several first rows

,price,href,title
0,£51.77,a-light-in-the-attic_1000/index.html,A Light in the Attic
1,£53.74,tipping-the-velvet_999/index.html,Tipping the Velvet
2,£50.10,soumission_998/index.html,Soumission
3,£47.82,sharp-objects_997/index.html,Sharp Objects
4,£54.23,sapiens-a-brief-history-of-humankind_996/index...,Sapiens: A Brief History of Humankind


Дополняем информацию о книге, добавляя описание книги и продуктовую информацию о стоимости/наличии в стоке. Для этого нужно будет воспользоваться второй ссылкой для каждой книги. Начнем с описания книги

In [17]:
# site for books parsing, description page of a book 'tipping-the-velvet'
url_new = 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html'
response_new = requests.get(url_new)
tree_new = BeautifulSoup(response_new.content, 'html.parser')# get the content into the tree

In [21]:
tree_new.find('article', {'class': 'product_page'}).find_all('p') # get description of this book in tags 
# the desciprion is in p tag, which does not have attributes

[<p class="price_color">£53.74</p>,
 <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock (20 available)
     
 </p>,
 <p class="star-rating One">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <!-- <small><a href="/catalogue/tipping-the-velvet_999/reviews/">
         
                 
                     0 customer reviews
                 
         </a></small>
          --> 
 
 
 <!-- 
     <a id="write_review" href="/catalogue/tipping-the-velvet_999/reviews/add/#addreview" class="btn btn-success btn-sm">
         Write a review
     </a>
 
  --></p>,
 <p>"Erotic and absorbing...Written with starling power."--"The New York Times Book Review " Nan King, an oyster girl, is captivated by the music hall phenomenon Kitty Butler, a male impersonator extraordinaire treading the boards in Canterbury. Through a friend at the box office, Nan manages to visit all her shows an

In [26]:
def get_description(link):
    # link - site link fragment
    # изготовили ссылку
    url = 'http://books.toscrape.com/catalogue/{}'.format(link)
    
    # сходили по ней
    response = requests.get(url)
    
    # построили дерево стр описания книги
    tree = BeautifulSoup(response.content, 'html.parser')
    
    # находим тег,  в котором находится описание. Оно в теге p. Есть несколько тегов p. 
    # сделаем так, чтобы самый последний из них - был тег с текстовым описанием, которое нам нужно
    temp = tree.find('article', {'class': 'product_page'})
    
    # у некоторых книг тестовое описание находится не в последнем теге p. Для них удалим тег ul, содерж. другой p
    if (tree.find('ul', {'class': 'row'})):
        temp.ul.replace_with("")
        
    # temp.find_all('p') - это массив. Берем последний тег p с помощью индекса -1
    # contents возвращает именно содержание тега в текстовой форме
    book_description = temp.find_all('p')[-1].contents[-1]
    
    return book_description
    

In [27]:
get_description("tipping-the-velvet_999/index.html") # description of a book 'tipping-the-velvet' as an example

'"Erotic and absorbing...Written with starling power."--"The New York Times Book Review " Nan King, an oyster girl, is captivated by the music hall phenomenon Kitty Butler, a male impersonator extraordinaire treading the boards in Canterbury. Through a friend at the box office, Nan manages to visit all her shows and finally meet her heroine. Soon after, she becomes Kitty\'s "Erotic and absorbing...Written with starling power."--"The New York Times Book Review " Nan King, an oyster girl, is captivated by the music hall phenomenon Kitty Butler, a male impersonator extraordinaire treading the boards in Canterbury. Through a friend at the box office, Nan manages to visit all her shows and finally meet her heroine. Soon after, she becomes Kitty\'s dresser and the two head for the bright lights of Leicester Square where they begin a glittering career as music-hall stars in an all-singing and dancing double act. At the same time, behind closed doors, they admit their attraction to each other 

In [28]:
# 'infa' variable contains book info: title, href, cost
# now we add description for each book in 'infa'

for i in range(0, len(infa)): 
    
    infa[i]['description'] = get_description(infa[i]['href']) # цикл проходит по всем книжкам в списке infa, добавляем в словарь описание c помощью вызова функции

In [29]:
df = pd.DataFrame(infa) # see that description was added as a column
df.head()

,price,href,title,description
0,£51.77,a-light-in-the-attic_1000/index.html,A Light in the Attic,It's hard to imagine a world without A Light i...
1,£53.74,tipping-the-velvet_999/index.html,Tipping the Velvet,"""Erotic and absorbing...Written with starling ..."
2,£50.10,soumission_998/index.html,Soumission,"Dans une France assez proche de la nôtre, un h..."
3,£47.82,sharp-objects_997/index.html,Sharp Objects,"WICKED above her hipbone, GIRL across her hear..."
4,£54.23,sapiens-a-brief-history-of-humankind_996/index...,Sapiens: A Brief History of Humankind,From a renowned historian comes a groundbreaki...


Теперь займемся добавлением инфо о книге

In [30]:
tree_new.find('table', {'class': 'table table-striped'}).find_all("tr") # get info about this book prices/availability in tags

[<tr>
 <th>UPC</th><td>90fa61229261140a</td>
 </tr>,
 <tr>
 <th>Product Type</th><td>Books</td>
 </tr>,
 <tr>
 <th>Price (excl. tax)</th><td>£53.74</td>
 </tr>,
 <tr>
 <th>Price (incl. tax)</th><td>£53.74</td>
 </tr>,
 <tr>
 <th>Tax</th><td>£0.00</td>
 </tr>,
 <tr>
 <th>Availability</th>
 <td>In stock (20 available)</td>
 </tr>,
 <tr>
 <th>Number of reviews</th>
 <td>0</td>
 </tr>]

In [31]:
for i in tree_new.find('table', {'class': 'table table-striped'}).find_all("tr"):
    print(i.th.text, i.td.text) # get text info from tags

UPC 90fa61229261140a
Product Type Books
Price (excl. tax) £53.74
Price (incl. tax) £53.74
Tax £0.00
Availability In stock (20 available)
Number of reviews 0


In [32]:
def get_info(link):
    # изготовили ссылку
    url = 'http://books.toscrape.com/catalogue/{}'.format(link)
    
    # сходили по ней
    response = requests.get(url)
    
    # построили дерево стр описания книги
    tree = BeautifulSoup(response.content, 'html.parser')
   
    info= {}
    
    # добавляем таблицу с инфо про цены, доступность
    for i in tree.find('table', {'class': 'table table-striped'}).find_all("tr"):
        info[i.th.text]=i.td.text # get text info from tags and put it into dictionary
    
    return info

In [33]:
get_info("tipping-the-velvet_999/index.html") # info of a book 'tipping-the-velvet' as an example

{'UPC': '90fa61229261140a',
 'Product Type': 'Books',
 'Price (excl. tax)': '£53.74',
 'Price (incl. tax)': '£53.74',
 'Tax': '£0.00',
 'Availability': 'In stock (20 available)',
 'Number of reviews': '0'}

In [34]:
for i in range(0, len(infa)):
    
    # infa[i]['info_in_dict']=get_info(infa[i]['href']) - это добавит нам целиком словарь с инфо в один столбец. 
    # в датафрейме будет +1 столб.
    
    infodict = get_info(infa[i]['href']) # info of a book in a dictionary
    
    # k here is a key of a dictionary 'infodict'
    
    for k in infodict:
        # infa[i] - i element of an 'infa' list. This element is a dictionary 
        # add into this dictionary key 'k' with value 'infodict[k]'
        infa[i][k] = infodict[k] # - это добавит нам каждый ключ словаря в свой столбец. На выходе в датафрейме будет +7 столб.
        

In [35]:
df = pd.DataFrame(infa)
df.head() # see that columns 'UPC', 'Product type', 'Price excl', 'Price incl', 'Tax', 'Availability', 'Reviews' were added

,price,href,title,description,UPC,Product Type,Price (excl. tax),Price (incl. tax),Tax,Availability,Number of reviews
0,£51.77,a-light-in-the-attic_1000/index.html,A Light in the Attic,It's hard to imagine a world without A Light i...,a897fe39b1053632,Books,£51.77,£51.77,£0.00,In stock (22 available),0
1,£53.74,tipping-the-velvet_999/index.html,Tipping the Velvet,"""Erotic and absorbing...Written with starling ...",90fa61229261140a,Books,£53.74,£53.74,£0.00,In stock (20 available),0
2,£50.10,soumission_998/index.html,Soumission,"Dans une France assez proche de la nôtre, un h...",6957f44c3847a760,Books,£50.10,£50.10,£0.00,In stock (20 available),0
3,£47.82,sharp-objects_997/index.html,Sharp Objects,"WICKED above her hipbone, GIRL across her hear...",e00eb4fd7b871a48,Books,£47.82,£47.82,£0.00,In stock (20 available),0
4,£54.23,sapiens-a-brief-history-of-humankind_996/index...,Sapiens: A Brief History of Humankind,From a renowned historian comes a groundbreaki...,4165285e1663650f,Books,£54.23,£54.23,£0.00,In stock (20 available),0


# Задание на семинаре


С сайта https://quotes.toscrape.com/ собрать все цитаты, авторов этих цитат, теги цитат. 
Тегов может быть несколько у одной цитаты

In [36]:
response2 = requests.get('https://quotes.toscrape.com/') # get content of  page

In [37]:
tree2 = BeautifulSoup(response2.content, 'html.parser') # put it into a tree of tags

In [38]:
quotes = tree2.find_all('div', {'class': 'quote'}) # find ALL div tags with class = quote. 'quotes' is a list

In [39]:
quotes[0] # first element of a list

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>

In [40]:
quotes[0].span.text # text of span tag

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

In [44]:
# we also need the author of a quote. We find small tag with class = author
authors = tree2.find_all('small', {'class': 'author'})

In [45]:
authors[0]

<small class="author" itemprop="author">Albert Einstein</small>

In [46]:
authors[0].text # text of small tag

'Albert Einstein'

In [47]:
# we also need tags of a quote. Each quote can have one or several tags
tags = tree2.find_all('div', {'class': 'tags'})

In [48]:
tags[0]

<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>

In [49]:
tags[0].find_all("a")

[<a class="tag" href="/tag/change/page/1/">change</a>,
 <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>,
 <a class="tag" href="/tag/thinking/page/1/">thinking</a>,
 <a class="tag" href="/tag/world/page/1/">world</a>]

In [50]:
for tag in tags[0].find_all("a"):
    print(tag.text) # we can get text of an a tags

change
deep-thoughts
thinking
world


Функция, которая соберет инфу со страницы: цитату, автора цитаты, теги этой цитаты

In [59]:
def get_page2(p):
    
    # изготовили ссылку
    url = 'https://quotes.toscrape.com/page/{}/'.format(p)
    
    # сходили по ней
    response = requests.get(url)
    
    # построили дерево 
    tree = BeautifulSoup(response.content, 'html.parser')
    
    # нашли тексты цитат, авторов этих цитат, теги
    quotes = tree.find_all('div', {'class': 'quote'}) # цитаты
    authors = tree.find_all('small', {'class': 'author'}) # авторы
    tags = tree.find_all('div', {'class': 'tags'}) # тэги
    
    quotes_infa = [ ]
    
    # сначала заполняем цитаты и их авторов. Каждой цитате соотв. только один автор, так что их количество совпадает
   
    for i in range(0, len(authors)):
        quotes_infa.append({'quote': quotes[i].span.text,
                     'author': authors[i].text,
                     'tags':[ ]})  
        
        # затем заполняем тэги
        a = [ ] # temporary list to keep tags of a quote
        
        for tag in tags[i].find_all("a"):
            a.append(tag.text)
        
        quotes_infa[i]['tags'] = a # we add this list of tags into 'quotes_infa' dictionary
        
                     
    return quotes_infa

In [60]:
get_page2(1) # as an example get the info from 'https://quotes.toscrape.com/page/1/'

[{'quote': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'author': 'Albert Einstein',
  'tags': ['change', 'deep-thoughts', 'thinking', 'world']},
 {'quote': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
  'author': 'J.K. Rowling',
  'tags': ['abilities', 'choices']},
 {'quote': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'author': 'Albert Einstein',
  'tags': ['inspirational', 'life', 'live', 'miracle', 'miracles']},
 {'quote': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
  'author': 'Jane Austen',
  'tags': ['aliteracy', 'books', 'classic', 'humor']},
 {'quote': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
  'author': 'Marilyn Monroe',
  'tags': 

In [61]:
infa2 = [ ]
# https://quotes.toscrape.com/ has 10 pages of quotes. Use range to collect info from these pages
for p in range(1,11):
    infa2.extend(get_page2(p))

In [62]:
len(infa2)

100

In [63]:
df2 = pd.DataFrame(infa2)
df2.head(100)

,quote,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,[better-life-empathy]
96,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,..."
97,“Never tell the truth to people who are not wo...,Mark Twain,[truth]
98,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational]
